In [2]:
# ==============================================================================
# V3 CELL 1 — THE SPACETIME COMPILER (3D MEASUREMENT NOISE)
# Run this as the FIRST cell in a clean "V3" Notebook
# ==============================================================================

import sys, subprocess, math, os, json, time
import numpy as np
import scipy.sparse as sp
import concurrent.futures
import multiprocessing
from datetime import datetime

try:
    import pymatching
except ImportError:
    print("Installing PyMatching...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pymatching", "scipy"], check=True)
    import pymatching

print("="*70)
print("V3 SHADOWMAP() PHYSICS ENGINE: 3D SHADOWMAP() SPACETIME MWPM (MEASUREMENT NOISE)")
print("="*70)

def build_3d_matrices(d, p):
    """
    Builds a 3D Spacetime Graph (d x d x T).
    - Space edges: Data qubit errors.
    - Time edges: Measurement errors.
    """
    T = d # Standard phenomenological scaling: Time steps = Distance
    H_rows, H_cols, L_cols = [], [], []
    weights, probs = [], []
    edge_idx = 0

    # Logical cut remains a vertical slice, but now errors cross it at ANY time step
    j_cut = d // 2
    if j_cut + 1 >= d: j_cut = max(0, d // 2 - 1)

    w = math.log((1-p)/p) if 0.0 < p < 0.5 else 1e-5

    # We stack 'T' measurement rounds on top of each other
    for t in range(T):
        for r in range(d):
            for c in range(d):
                u = t * (d * d) + r * d + c

                # 1. Space-Horizontal (Data Qubit Error)
                if c + 1 < d:
                    v = t * (d * d) + r * d + (c + 1)
                    H_rows.extend([u, v]); H_cols.extend([edge_idx, edge_idx])
                    weights.append(w); probs.append(p)
                    # Data errors persist and accumulate, so any cut-crossing flips final parity
                    if c == j_cut: L_cols.append(edge_idx)
                    edge_idx += 1

                # 2. Space-Vertical (Data Qubit Error)
                if r + 1 < d:
                    v = t * (d * d) + (r + 1) * d + c
                    H_rows.extend([u, v]); H_cols.extend([edge_idx, edge_idx])
                    weights.append(w); probs.append(p)
                    edge_idx += 1

                # 3. Virtual Spatial Boundaries (Left and Right)
                if c == 0 or c == d - 1:
                    H_rows.append(u); H_cols.append(edge_idx)
                    weights.append(0.0); probs.append(0.0)
                    edge_idx += 1

                # 4. Time Edges (Measurement Errors)
                if t + 1 < T:
                    v = (t + 1) * (d * d) + r * d + c
                    H_rows.extend([u, v]); H_cols.extend([edge_idx, edge_idx])
                    weights.append(w); probs.append(p)
                    edge_idx += 1
                else:
                    # Temporal Boundary (Assume perfect measurement at final round)
                    H_rows.append(u); H_cols.append(edge_idx)
                    weights.append(w); probs.append(p)
                    edge_idx += 1

    num_nodes = T * d * d
    H = sp.csc_matrix(([1]*len(H_rows), (H_rows, H_cols)), shape=(num_nodes, edge_idx), dtype=np.uint8)
    L = sp.csc_matrix(([1]*len(L_cols), ([0]*len(L_cols), L_cols)), shape=(1, edge_idx), dtype=np.uint8)

    return H, L, np.array(weights), np.array(probs)

def wilson_ci(k, n, z=1.96):
    if n == 0: return (0.0, 0.0)
    phat = k / n
    denom = 1.0 + (z*z)/n
    center = (phat + (z*z)/(2*n)) / denom
    half = (z * math.sqrt((phat*(1-phat) + (z*z)/(4*n)) / n)) / denom
    return (max(0.0, center - half), min(1.0, center + half))

def run_3d_chunk(args):
    d, p, trials, seed = args
    rng = np.random.default_rng(seed)

    H, L, weights, probs = build_3d_matrices(d, p)
    num_edges = len(probs)

    matcher = pymatching.Matching.from_check_matrix(H, weights=weights, faults_matrix=L)

    noise_matrix = (rng.random((trials, num_edges)) < probs).astype(np.uint8)

    H_csr, L_csr = H.tocsr(), L.tocsr()
    syndromes = np.asarray((H_csr @ noise_matrix.T).T % 2, dtype=np.uint8)
    actual_obs = np.asarray((L_csr @ noise_matrix.T).T % 2, dtype=np.uint8)[:, 0]

    pred_obs = matcher.decode_batch(syndromes)[:, 0]
    failures = int(np.sum(pred_obs != actual_obs))

    return failures

if __name__ == "__main__":
    NUM_CORES = min(44, multiprocessing.cpu_count())
    print(f"Detected {NUM_CORES} CPU Cores. Initializing 3D Multiprocessing Pool...")

    distances = [5, 7, 9]
    # 3D Phenomenological Threshold is ~2.9% (0.029)
    p_grid = [0.020, 0.025, 0.029, 0.035, 0.040]
    total_trials = 20000

    results = {}
    t0 = time.time()

    with concurrent.futures.ProcessPoolExecutor(max_workers=NUM_CORES) as executor:
        for d in distances:
            results[d] = {}
            for p in p_grid:
                chunk_size = max(1, total_trials // NUM_CORES)
                actual_trials = chunk_size * NUM_CORES
                tasks = [(d, p, chunk_size, hash((d, p, i, 42)) % (2**32)) for i in range(NUM_CORES)]

                failures = sum(executor.map(run_3d_chunk, tasks))
                rate = failures / actual_trials
                lo, hi = wilson_ci(failures, actual_trials)
                results[d][p] = {"rate": rate, "ci95": (lo, hi), "k": failures, "n": actual_trials}

            print(f"Completed 3D sweep for d={d:>2}")

    elapsed = time.time() - t0

    print("\n" + "="*70)
    print("SHADOWMAP() DERIVED 3D SPACETIME PHASE TRANSITION (EXPECTED CROSSING ~0.029)")
    print("="*70)
    header = "d \\ p | " + " | ".join([f"{p:>10.3f}" for p in p_grid])
    print(header)
    print("-" * len(header))
    for d in distances:
        row = []
        for p in p_grid:
            r = results[d][p]["rate"]
            lo, hi = results[d][p]["ci95"]
            row.append(f"{r:.4f}[{lo:.3f},{hi:.3f}]")
        print(f"{d:>5} | " + " | ".join(row))
    print("="*70)

    total_decodes = len(distances) * len(p_grid) * actual_trials
    print(f"Total SHADOWPAP() DERIVED 3D Decodes Performed: {total_decodes:,}")
    print(f"Total Elapsed Time:         {elapsed:.2f} seconds")
    print("="*70)

V3 SHADOWMAP() PHYSICS ENGINE: 3D SHADOWMAP() SPACETIME MWPM (MEASUREMENT NOISE)
Detected 8 CPU Cores. Initializing 3D Multiprocessing Pool...
Completed 3D sweep for d= 5
Completed 3D sweep for d= 7
Completed 3D sweep for d= 9

SHADOWMAP() DERIVED 3D SPACETIME PHASE TRANSITION (EXPECTED CROSSING ~0.029)
d \ p |      0.020 |      0.025 |      0.029 |      0.035 |      0.040
----------------------------------------------------------------------
    5 | 0.0646[0.061,0.068] | 0.1031[0.099,0.107] | 0.1439[0.139,0.149] | 0.2094[0.204,0.215] | 0.2541[0.248,0.260]
    7 | 0.0681[0.065,0.072] | 0.1126[0.108,0.117] | 0.1678[0.163,0.173] | 0.2470[0.241,0.253] | 0.3171[0.311,0.324]
    9 | 0.0716[0.068,0.075] | 0.1238[0.119,0.128] | 0.1847[0.179,0.190] | 0.2933[0.287,0.300] | 0.3712[0.365,0.378]
Total SHADOWPAP() DERIVED 3D Decodes Performed: 300,000
Total Elapsed Time:         1.91 seconds


In [3]:
# ==============================================================================
# V3 CELL 2 — 3D SPACETIME CO-DESIGN (BROKEN MEASUREMENT SENSORS)
# Run this BELOW V3 Cell 1
# ==============================================================================

import pymatching
import scipy.sparse as sp
import scipy.stats
import numpy as np
import random, time, math

print("="*70)
print("V3 CELL 2: 3D SPACETIME CO-DESIGN (BROKEN MEASUREMENT SENSORS)")
print("="*70)

d = 7
T = 7
trials = 50000

# 1. The 3D Environment: Sensor at Column 3 is heavily damaged.
p_space = 0.02
p_time_good = 0.02
p_time_bad = 0.15
bad_sensor_col = 3

print(f"Fabricating 3D Spacetime block (d={d}, T={T})...")
print(f" -> Healthy Space/Time Error: {p_space*100}%")
print(f" -> BROKEN SENSOR (Col {bad_sensor_col}): {p_time_bad*100}% Measurement Error")

H_rows, H_cols, L_cols, probs = [], [], [], []
edge_types, edge_cols = [], []
edge_idx = 0
j_cut = d // 2

for t in range(T):
    for r in range(d):
        for c in range(d):
            u = t * (d * d) + r * d + c

            # Space-Horizontal
            if c + 1 < d:
                v = t * (d * d) + r * d + (c + 1)
                H_rows.extend([u, v]); H_cols.extend([edge_idx, edge_idx])
                probs.append(p_space); edge_types.append('S'); edge_cols.append(c)
                if c == j_cut: L_cols.append(edge_idx)
                edge_idx += 1

            # Space-Vertical
            if r + 1 < d:
                v = t * (d * d) + (r + 1) * d + c
                H_rows.extend([u, v]); H_cols.extend([edge_idx, edge_idx])
                probs.append(p_space); edge_types.append('S'); edge_cols.append(c)
                edge_idx += 1

            # Virtual Spatial Boundaries
            if c == 0 or c == d - 1:
                H_rows.append(u); H_cols.append(edge_idx)
                probs.append(0.0); edge_types.append('B'); edge_cols.append(c)
                edge_idx += 1

            # Time Edges (MEASUREMENT SENSORS)
            if t + 1 < T:
                v = (t + 1) * (d * d) + r * d + c
                H_rows.extend([u, v]); H_cols.extend([edge_idx, edge_idx])

                # INJECT THE BROKEN SENSOR
                p_t = p_time_bad if c == bad_sensor_col else p_time_good
                probs.append(p_t); edge_types.append('T'); edge_cols.append(c)
                edge_idx += 1
            else:
                # Perfect final measurement
                H_rows.append(u); H_cols.append(edge_idx)
                probs.append(0.0); edge_types.append('B_T'); edge_cols.append(c)
                edge_idx += 1

num_nodes = T * d * d
H = sp.csc_matrix(([1]*len(H_rows), (H_rows, H_cols)), shape=(num_nodes, edge_idx), dtype=np.uint8)
L = sp.csc_matrix(([1]*len(L_cols), ([0]*len(L_cols), L_cols)), shape=(1, edge_idx), dtype=np.uint8)
probs = np.array(probs)

print(f"Generating {trials:,} Spacetime CRN Trials for Train and Test...")
H_csr, L_csr = H.tocsr(), L.tocsr()
num_edges = len(probs)

def make_crn(seed):
    rng = np.random.default_rng(seed)
    noise = (rng.random((trials, num_edges)) < probs).astype(np.uint8)
    syn = np.asarray((H_csr @ noise.T).T % 2, dtype=np.uint8)
    obs = np.asarray((L_csr @ noise.T).T % 2, dtype=np.uint8)[:, 0]
    return syn, obs

syn_train, obs_train = make_crn(101)
syn_test,  obs_test  = make_crn(202)

is_S = np.array([typ == 'S' for typ in edge_types])
is_T = np.array([typ == 'T' for typ in edge_types])
is_B = np.array([(typ == 'B' or typ == 'B_T') for typ in edge_types])
col_idx_arr = np.array(edge_cols)

def evaluate_3d_map(w_space, w_time_arr, dataset="train"):
    weights = np.zeros(num_edges, dtype=np.float32)
    weights[is_S] = max(0.01, w_space)
    weights[is_T] = np.maximum(0.01, w_time_arr[col_idx_arr[is_T]])
    weights[is_B] = 0.0

    matcher = pymatching.Matching.from_check_matrix(H, weights=weights, faults_matrix=L)

    if dataset == "train":
        fails = (matcher.decode_batch(syn_train)[:, 0] != obs_train)
    else:
        fails = (matcher.decode_batch(syn_test)[:, 0] != obs_test)

    return int(np.sum(fails)), fails

print("\nEvaluating Naive 3D Baseline (Uniform weights = 1.0)...")
base_k_train, _ = evaluate_3d_map(1.0, np.ones(d), "train")
print(f" -> Baseline Train Rate: {base_k_train/trials:.5f}\n")

pop_size = 15
generations = 10
retain = 5

population = [{'w_S': random.uniform(0.8, 1.2), 'w_T': np.random.uniform(0.8, 1.2, d)} for _ in range(pop_size)]

print("Starting 3D SHADOWMAP Spacetime Optimizer...")
t0 = time.time()
best_history = []

for gen in range(generations):
    scored = []
    for theta in population:
        fails, _ = evaluate_3d_map(theta['w_S'], theta['w_T'], "train")
        scored.append((fails / trials, theta))

    scored.sort(key=lambda x: x[0])
    survivors = scored[:retain]
    best_rate, best_theta = survivors[0]
    best_history.append((best_rate, best_theta))

    print(f"GEN {gen+1:02d} | Fail: {best_rate:.5f} | w_Time_Normal: {best_theta['w_T'][0]:.2f} | w_Time_DEFECT: {best_theta['w_T'][bad_sensor_col]:.2f}")

    new_pop = [s[1] for s in survivors]
    while len(new_pop) < pop_size:
        parent = random.choice(survivors)[1]
        child = {
            'w_S': max(0.01, parent['w_S'] + random.uniform(-0.15, 0.15)),
            'w_T': np.maximum(0.01, parent['w_T'] + np.random.normal(0, 0.15, d))
        }
        new_pop.append(child)
    population = new_pop

opt_theta = best_history[-1][1]
elapsed = time.time() - t0

base_k_test, base_fails = evaluate_3d_map(1.0, np.ones(d), "test")
opt_k_test, opt_fails   = evaluate_3d_map(opt_theta['w_S'], opt_theta['w_T'], "test")

# Theoretical Max-Likelihood Map
th_w_S = math.log((1 - p_space) / p_space)
th_w_T = np.zeros(d)
for c in range(d):
    p_c = p_time_bad if c == bad_sensor_col else p_time_good
    th_w_T[c] = math.log((1 - p_c) / p_c)
th_k_test, th_fails = evaluate_3d_map(th_w_S, th_w_T, "test")

n01 = int(np.sum((base_fails == True) & (opt_fails == False)))
n10 = int(np.sum((base_fails == False) & (opt_fails == True)))
chi2_stat = (abs(n01 - n10) - 1)**2 / (n01 + n10) if (n01 + n10) > 0 else 0
p_val = scipy.stats.chi2.sf(chi2_stat, 1) if (n01 + n10) > 0 else 1.0

print("\n" + "="*70)
print(f"3D SPATIAL-TEMPORAL OPTIMIZATION COMPLETE IN {elapsed:.2f}s")
print("="*70)
print(f"TEST Baseline Failure Rate: {base_k_test/trials:.5f}")
print(f"TEST SHADOWMAP Optimized:   {opt_k_test/trials:.5f}  <--- SENSOR MAPPED")
print(f"TEST Theoretical Limit:     {th_k_test/trials:.5f}")
print("-" * 70)
print("STATISTICAL SIGNIFICANCE (McNemar Test):")
print(f"  Failures avoided by AI: {n01:,}")
print(f"  New failures caused:    {n10:,}")
print(f"  p-value:                {p_val:.2e}")
print("="*70)

print("\nSHADOWMAP() DISCOVERED SENSOR MAP (Time Weights by Column):")
print("Col | True Meas. Error | SHADOWMAP (Scaled) | Theoretical Opt")
print("-" * 65)

scale_factor = th_w_S / opt_theta['w_S']
scaled_w_T = opt_theta['w_T'] * scale_factor

for c in range(d):
    p_c = p_time_bad if c == bad_sensor_col else p_time_good
    tag = "<-- BROKEN SENSOR DISCOVERED" if c == bad_sensor_col else ""
    print(f"{c:>3} | {p_c:>16.2f} | {scaled_w_T[c]:^18.3f} | {th_w_T[c]:^15.3f} {tag}")
print("="*70)

V3 CELL 2: 3D SPACETIME CO-DESIGN (BROKEN MEASUREMENT SENSORS)
Fabricating 3D Spacetime block (d=7, T=7)...
 -> Healthy Space/Time Error: 2.0%
 -> BROKEN SENSOR (Col 3): 15.0% Measurement Error
Generating 50,000 Spacetime CRN Trials for Train and Test...

Evaluating Naive 3D Baseline (Uniform weights = 1.0)...
 -> Baseline Train Rate: 0.28000

Starting 3D SHADOWMAP Spacetime Optimizer...
GEN 01 | Fail: 0.24708 | w_Time_Normal: 1.05 | w_Time_DEFECT: 0.95
GEN 02 | Fail: 0.24328 | w_Time_Normal: 1.04 | w_Time_DEFECT: 0.62
GEN 03 | Fail: 0.24168 | w_Time_Normal: 1.10 | w_Time_DEFECT: 0.56
GEN 04 | Fail: 0.23832 | w_Time_Normal: 1.34 | w_Time_DEFECT: 0.46
GEN 05 | Fail: 0.23832 | w_Time_Normal: 1.34 | w_Time_DEFECT: 0.46
GEN 06 | Fail: 0.23832 | w_Time_Normal: 1.34 | w_Time_DEFECT: 0.46
GEN 07 | Fail: 0.23830 | w_Time_Normal: 1.23 | w_Time_DEFECT: 0.51
GEN 08 | Fail: 0.23804 | w_Time_Normal: 1.35 | w_Time_DEFECT: 0.61
GEN 09 | Fail: 0.23804 | w_Time_Normal: 1.35 | w_Time_DEFECT: 0.61
GEN 10